In [1]:
import difflib
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.reference.constants import COMPONENT_TYPE_MAP, POWER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

In [2]:
from libs.interfaces.rolodex_interface import rolodex_load

In [4]:
data = rolodex_load(
    "inspection", 
    links=[
        "component", 
        "unit", 
        "site", 
        "organization"
    ], 
    filters=[
        {
            "type": "unit", 
            "key": "portal.name", 
            "value": "liq"
        }
    ], 
    ids=[], 
    tags={
        "organization": "georgia-pacific-a1ec53"
    }, 
    limit=100000
)

In [27]:
list(pd.json_normalize(data[0]).columns)

['entry.type',
 'entry.data.portal.id',
 'entry.data.portal.date',
 'entry.data.portal.demo',
 'entry.data.portal.name',
 'entry.data.portal.slug',
 'entry.data.portal.notes',
 'entry.data.portal.job_id',
 'entry.data.portal.status',
 'entry.data.portal.read_only',
 'entry.data.portal.show_excel',
 'entry.data.portal.sort_order',
 'entry.data.portal.data_import',
 'entry.data.portal.external_id',
 'entry.data.portal.zero_offset',
 'entry.data.portal.component_id',
 'entry.data.portal.inspection_type',
 'entry.data.portal.ndt_method_type',
 'entry.data.portal.target_release_time',
 'entry.data.portal.sales_robotic_service_id',
 'entry.id',
 'entry.tags.job',
 'entry.tags.site',
 'entry.tags.unit',
 'entry.tags.component',
 'entry.tags.inspection',
 'entry.tags.organization',
 'links.component.type',
 'links.component.data.portal.id',
 'links.component.data.portal.lat',
 'links.component.data.portal.lng',
 'links.component.data.portal.name',
 'links.component.data.portal.slug',
 'links.c

In [44]:
data = rolodex_load(
    "inspection", 
    links=[
        "component", 
        "unit", 
        "site", 
        "organization"
    ], 
    filters=[
        {
            "type": "component", 
            "key": "portal.component_type", 
            "value": "2"
        }
    ], 
    ids=[], 
    tags={
    }, 
    limit=100000
)

In [45]:
tank_shell_inspections_df = pd.json_normalize(data)

In [46]:
tank_shell_inspections_df.shape

(1667, 160)

In [47]:
tank_shell_inspections_df.head()

,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,...,links.unit.data.opportunity_low,links.unit.data.opportunity_high,links.unit.data.opportunity_size,links.site.data.capacity_kw,links.site.tags.region,links.site.data.opportunity_mwh,links.site.data.opportunity_size,links.site.data.opportunity_tubes,links.site.data.opportunity_dollars,links.unit.data.values.manufactured_date
0,inspection,3866,2022-09-27,0,None,20220927-c487ba,,1295,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,inspection,3867,2022-09-28,0,None,20220927-f89976,,1295,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,inspection,3868,2022-09-28,0,None,20220928-eedd93,,1295,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,inspection,3873,2022-09-29,0,None,20220928-2e1302,,1295,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,inspection,3874,2022-09-29,0,None,20220928-ce860f,,1295,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
tank_shell_inspections_df['entry.data.portal.date'].min()

'2017-04-24'

In [49]:
tank_shell_inspections_df['entry.data.portal.date'].max()

'2023-01-31'

In [50]:
completed_tank_shell_inspections_df = tank_shell_inspections_df[tank_shell_inspections_df['entry.data.portal.status']=='completed']

In [51]:
completed_tank_shell_inspections_df['entry.data.portal.date'].max()

'2023-01-17'

In [57]:
PAPER_CUSTOMERS = [
    'Clearwater Paper',
    'Domtar',
    'Evergreen Packaging',
    'Georgia Pacific',
    'Huber Engineered Woods',
    'International Paper',
    'Packaging Corporation of America',
    'Pixelle Specialty Solutions',
    'Rayonier Advanced Materials',
    'Westrock'
]

In [59]:
paper_tank_shell_inspections_df = completed_tank_shell_inspections_df[completed_tank_shell_inspections_df['links.organization.data.portal.name'].isin(PAPER_CUSTOMERS)]

In [60]:
paper_tank_shell_inspections_df.head()

,entry.type,entry.data.portal.id,entry.data.portal.date,entry.data.portal.demo,entry.data.portal.name,entry.data.portal.slug,entry.data.portal.notes,entry.data.portal.job_id,entry.data.portal.status,entry.data.portal.read_only,...,links.unit.data.opportunity_low,links.unit.data.opportunity_high,links.unit.data.opportunity_size,links.site.data.capacity_kw,links.site.tags.region,links.site.data.opportunity_mwh,links.site.data.opportunity_size,links.site.data.opportunity_tubes,links.site.data.opportunity_dollars,links.unit.data.values.manufactured_date
9,inspection,3647,2022-06-16,0,None,20220616-ff6ae3,,1048,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,inspection,4101,2022-12-14,0,None,20221214-99f216,,1350,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,inspection,1606,2017-07-28,0,None,20170728-5ade01,,394,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,inspection,719,2019-08-27,0,None,20190827-960bc9,,182,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
269,inspection,724,2019-08-30,0,None,20190830-936911,,185,completed,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
sliced_inspection_df = paper_tank_shell_inspections_df.loc[:, ('entry.data.portal.slug', 'links.component.data.portal.wall_nominal_thickness')].reset_index()

In [62]:
sliced_inspection_df.rename(
    columns={
        'entry.data.portal.slug': 'slug',	
        'links.component.data.portal.wall_nominal_thickness': 'nominal',
    }, 
    inplace=True
)
sliced_inspection_df.drop(columns={'index'}, inplace=True)

In [63]:
sliced_inspection_df.head()

,slug,nominal
0,20220616-ff6ae3,1.50
1,20221214-99f216,0.25
2,20170728-5ade01,0.50
3,20190827-960bc9,0.78
4,20190830-936911,0.78


In [64]:
sliced_inspection_df.nominal.describe()

count    185.000000
mean       0.528835
std        0.382558
min        0.000000
25%        0.250000
50%        0.375000
75%        0.625000
max        1.750000
Name: nominal, dtype: float64

In [65]:
clean_nominals_df = sliced_inspection_df[
    ~sliced_inspection_df['nominal'].isna() &
    (sliced_inspection_df['nominal']!=0) &
    (sliced_inspection_df['nominal'] < 1) &
    (sliced_inspection_df['nominal'] > .1)
]

In [66]:
clean_nominals_df.nominal.describe()

count    164.000000
mean       0.430698
std        0.201219
min        0.187500
25%        0.250000
50%        0.375000
75%        0.500000
max        0.906500
Name: nominal, dtype: float64

In [68]:
clean_nominals_df.reset_index(inplace=True)

In [69]:
histogram_df, hist_error_slug_list = get_thickness_histogram(clean_nominals_df)

0/164
10/164
20/164
30/164
40/164
50/164
Error: No plots record on req.json ...
60/164
70/164
80/164
90/164
100/164
110/164
Error: No plots record on req.json ...
120/164
130/164
140/164
150/164
Error: No plots record on req.json ...
Error: No plots record on req.json ...
160/164
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...
Error: No plots record on req.json ...


In [70]:
histogram_df.shape

(156, 25)

In [71]:
len(hist_error_slug_list)

8

In [72]:
histogram_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,...,Tubes w 90% Loss,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,Bins w 40% Loss,Bins w 50% Loss,Bins w 60% Loss,Bins w 70% Loss,Bins w 80% Loss,Bins w 90% Loss
0,20221214-99f216,0.25,0.220400,0.295800,28,2043,0,3,0,0,...,0,3,0,0,0,0,0,0,0,0
1,20170728-5ade01,0.50,0.379000,0.546000,30,600,0,10,5,0,...,0,250,25,0,0,0,0,0,0,0
2,20190827-960bc9,0.78,0.738082,1.037543,74,4641,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20190830-936911,0.78,0.835009,0.904185,38,1454,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20190815-3dca29,0.30,0.093000,0.459300,12,636,189,12,12,4,...,0,385,110,13,12,11,3,0,0,0


In [73]:
comb_df = histogram_df.merge(paper_tank_shell_inspections_df, how='left', left_on='slug', right_on='entry.data.portal.slug')

In [74]:
comb_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Tubes w 10% Loss,Tubes w 20% Loss,Tubes w 30% Loss,...,links.unit.data.opportunity_low,links.unit.data.opportunity_high,links.unit.data.opportunity_size,links.site.data.capacity_kw,links.site.tags.region,links.site.data.opportunity_mwh,links.site.data.opportunity_size,links.site.data.opportunity_tubes,links.site.data.opportunity_dollars,links.unit.data.values.manufactured_date
0,20221214-99f216,0.25,0.220400,0.295800,28,2043,0,3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20170728-5ade01,0.50,0.379000,0.546000,30,600,0,10,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20190827-960bc9,0.78,0.738082,1.037543,74,4641,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20190830-936911,0.78,0.835009,0.904185,38,1454,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20190815-3dca29,0.30,0.093000,0.459300,12,636,189,12,12,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
list(comb_df.columns)

['slug',
 'nominal',
 'min_t',
 'max_t',
 'tubes_inspected',
 'bins_collected',
 'crits_per10k',
 'Tubes w 10% Loss',
 'Tubes w 20% Loss',
 'Tubes w 30% Loss',
 'Tubes w 40% Loss',
 'Tubes w 50% Loss',
 'Tubes w 60% Loss',
 'Tubes w 70% Loss',
 'Tubes w 80% Loss',
 'Tubes w 90% Loss',
 'Bins w 10% Loss',
 'Bins w 20% Loss',
 'Bins w 30% Loss',
 'Bins w 40% Loss',
 'Bins w 50% Loss',
 'Bins w 60% Loss',
 'Bins w 70% Loss',
 'Bins w 80% Loss',
 'Bins w 90% Loss',
 'entry.type',
 'entry.data.portal.id',
 'entry.data.portal.date',
 'entry.data.portal.demo',
 'entry.data.portal.name',
 'entry.data.portal.slug',
 'entry.data.portal.notes',
 'entry.data.portal.job_id',
 'entry.data.portal.status',
 'entry.data.portal.read_only',
 'entry.data.portal.show_excel',
 'entry.data.portal.sort_order',
 'entry.data.portal.data_import',
 'entry.data.portal.external_id',
 'entry.data.portal.zero_offset',
 'entry.data.portal.component_id',
 'entry.data.portal.inspection_type',
 'entry.data.portal.ndt_met

In [85]:
trimmed_comb_df = comb_df.loc[:,(
     'slug',
     'nominal',
     'min_t',
     'max_t',
     'tubes_inspected',
     'bins_collected',
     'crits_per10k',
     'Bins w 10% Loss',
     'Bins w 20% Loss',
     'Bins w 30% Loss',
     'Bins w 40% Loss',
     'Bins w 50% Loss',
     'Bins w 60% Loss',
     'Bins w 70% Loss',
     'Bins w 80% Loss',
     'Bins w 90% Loss',
     'entry.data.portal.date',
     'entry.data.portal.demo',
     'entry.tags.job',
     'entry.tags.site',
     'entry.tags.unit',
     'entry.tags.component',
     'entry.tags.organization',
     'links.component.data.portal.shell_diameter',
     'links.unit.data.portal.name',
     'links.site.data.portal.name',
     'links.organization.data.portal.name',
)]

In [86]:
trimmed_comb_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,...,entry.data.portal.demo,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.organization,links.component.data.portal.shell_diameter,links.unit.data.portal.name,links.site.data.portal.name,links.organization.data.portal.name
0,20221214-99f216,0.25,0.220400,0.295800,28,2043,0,3,0,0,...,0,job-7cf4d9,brewton-mill-fb51be,1-high-yield-brown-stock-washer-ff87e8,tank-shell-b6bc56,georgia-pacific-a1ec53,12.0,#1 High Yield Brown Stock Washer,Brewton Mill,Georgia Pacific
1,20170728-5ade01,0.50,0.379000,0.546000,30,600,0,250,25,0,...,0,job-8298cf,brewton-mill-fb51be,sulfide-tank-f9c999,tank-shell-b2d1a2,georgia-pacific-a1ec53,20.0,(7303) Sulfide Tank,Brewton Mill,Georgia Pacific
2,20190827-960bc9,0.78,0.738082,1.037543,74,4641,0,0,0,0,...,0,job-ee7423,brewton-mill-fb51be,2205-duplex-digester-e1bb76,tank-wall-ce2e2c,georgia-pacific-a1ec53,12.0,(2303) Digester #3,Brewton Mill,Georgia Pacific
3,20190830-936911,0.78,0.835009,0.904185,38,1454,0,0,0,0,...,0,job-060283,brewton-mill-fb51be,2205-duplex-digester-1-4aa012,digester-shell-eddea7,georgia-pacific-a1ec53,12.0,(2301) Digester #1,Brewton Mill,Georgia Pacific
4,20190815-3dca29,0.30,0.093000,0.459300,12,636,189,385,110,13,...,0,job-234ecd,brunswick-cellulose-305428,firewater-tank-54073-8099f2,tank-wall-a968ad,georgia-pacific-a1ec53,40.0,Firewater Tank 54073,Brunswick Cellulose,Georgia Pacific


In [79]:
trimmed_comb_df.sort_values(by=['crits_per10k'], ascending=False)

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,...,entry.data.portal.date,entry.data.portal.demo,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.organization,links.component.data.portal.shell_diameter,links.site.data.portal.name,links.organization.data.portal.name
77,20210607-beeac6,0.9065,0.29580,0.42920,11,160,10000,160,160,160,...,2021-06-07,0,job-2c9487,pixelle-spring-grove-3cfd03,blow-lines-819c68,digester-1-line-55a8dc,pixelle-specialty-solutions-599e62,0.0833,Spring Grove,Pixelle Specialty Solutions
71,20210607-3ede00,0.9065,0.22736,0.59972,7,87,9885,87,87,87,...,2021-06-07,0,job-2c9487,pixelle-spring-grove-3cfd03,blow-lines-819c68,digester-7-line-a87241,pixelle-specialty-solutions-599e62,0.0833,Spring Grove,Pixelle Specialty Solutions
76,20210607-b130a5,0.9065,0.31784,1.89892,12,201,9652,200,199,198,...,2021-06-07,0,job-2c9487,pixelle-spring-grove-3cfd03,blow-lines-819c68,digester-8-line-8cb81f,pixelle-specialty-solutions-599e62,0.0833,Spring Grove,Pixelle Specialty Solutions
73,20210607-e35262,0.9065,0.11800,0.71920,12,288,9167,288,288,264,...,2021-06-07,0,job-2c9487,pixelle-spring-grove-3cfd03,blow-lines-819c68,digester-6-line-d42cf0,pixelle-specialty-solutions-599e62,0.0833,Spring Grove,Pixelle Specialty Solutions
78,20210607-bf826d,0.9065,0.31320,0.68904,11,149,8859,149,149,146,...,2021-06-07,0,job-2c9487,pixelle-spring-grove-3cfd03,blow-lines-819c68,digester-2-3-blow-line-3ae601,pixelle-specialty-solutions-599e62,0.0833,Spring Grove,Pixelle Specialty Solutions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20210908-830660,0.2500,0.37120,1.06488,11,2072,0,0,0,0,...,2021-09-08,0,job-5b7c12,international-falls-kraft-pulp-and-paper-mi-0a...,blow-line-circuit-7b3db1,digester-lines-5678-32312b,packaging-corporation-of-america-7f9e0d,1.0000,International Falls Kraft Pulp and Paper Mill,Packaging Corporation of America
48,20201207-ac24e1,0.3125,0.31320,0.38900,102,3375,0,0,0,0,...,2020-12-07,0,job-b53668,brewton-mill-fb51be,c1-3-24-e4d17b,c1-3-124-1e4251,georgia-pacific-a1ec53,8.0000,Brewton Mill,Georgia Pacific
49,20201207-50ff61,0.3750,0.32816,0.39109,23,206,0,2,0,0,...,2020-12-07,0,job-b53668,brewton-mill-fb51be,c1-3-112-6705bd,c1-3-112-head-2-a39429,georgia-pacific-a1ec53,8.0000,Brewton Mill,Georgia Pacific
95,20210908-042c78,0.2500,0.32480,0.65772,13,453,0,0,0,0,...,2021-09-08,0,job-deda8d,international-falls-kraft-pulp-and-paper-mi-0a...,blow-line-circuit-7b3db1,digester-lines-4-56fe11,packaging-corporation-of-america-7f9e0d,1.0000,International Falls Kraft Pulp and Paper Mill,Packaging Corporation of America


In [80]:
set(trimmed_comb_df['entry.data.portal.demo'])

{0}

In [87]:
set(trimmed_comb_df['links.unit.data.portal.name'])

{'#1 Acid Tank',
 '#1 Boiler Stack',
 '#1 Brown Stock Washer',
 '#1 HD Hardwood (16850)',
 '#1 HD Pine Storage (85564)',
 '#1 High Yield Brown Stock Washer',
 '#1 M&D',
 '#2 Brown Stock Washer',
 '#2 Green Liquor Clarifier',
 '#2 HD Pine (16868)',
 '#2 HD Pine Storage (85565)',
 '#2 High Yield Brown Stock Washer',
 '#2 M&D',
 '#3 Black Liquor Storage Tank',
 '#3 Brown Stock Washer',
 '#3 HD Pine Storage (85600)',
 '#3 High Yield Brown Stock Washer',
 '#4 Black Liquor Storage Tank',
 '#4 Brown Stock Washer',
 '#4 High Yield Brown Stock Washer',
 '#4 Weak Black Liquor Tank',
 '#5 Brown Stock Washer',
 '(12305) Neutralization Acid Tank',
 '(1317) Cold Demin Water Storage',
 '(14304) 93% Sulfuric Acid Tank',
 '(2301) Digester #1',
 '(2302) Digester #2',
 '(2303) Digester #3',
 '(2307) Blow Tank',
 '(3307) Tank',
 '(3309) #5 Filtrate Tank',
 '(3385) Tank North BMP Spill',
 '(4311) Sulfuric Acid Tank',
 '(7303) Sulfide Tank',
 '(8302) Tank',
 '(8310) Tank',
 '(A2301) #6 Batch Digester',
 '(A

In [90]:
tanks_df = trimmed_comb_df[trimmed_comb_df['links.unit.data.portal.name'].apply(lambda x: 'tank' in x.lower())]

In [92]:
tanks_df.head()

,slug,nominal,min_t,max_t,tubes_inspected,bins_collected,crits_per10k,Bins w 10% Loss,Bins w 20% Loss,Bins w 30% Loss,...,entry.data.portal.demo,entry.tags.job,entry.tags.site,entry.tags.unit,entry.tags.component,entry.tags.organization,links.component.data.portal.shell_diameter,links.unit.data.portal.name,links.site.data.portal.name,links.organization.data.portal.name
1,20170728-5ade01,0.50,0.37900,0.546000,30,600,0,250,25,0,...,0,job-8298cf,brewton-mill-fb51be,sulfide-tank-f9c999,tank-shell-b2d1a2,georgia-pacific-a1ec53,20.0,(7303) Sulfide Tank,Brewton Mill,Georgia Pacific
4,20190815-3dca29,0.30,0.09300,0.459300,12,636,189,385,110,13,...,0,job-234ecd,brunswick-cellulose-305428,firewater-tank-54073-8099f2,tank-wall-a968ad,georgia-pacific-a1ec53,40.0,Firewater Tank 54073,Brunswick Cellulose,Georgia Pacific
5,20190930-32a092,0.25,0.14100,0.274000,4,56,179,23,11,4,...,0,job-95bc4e,brewton-mill-fb51be,blend-tanka8306-ff829b,tank-shell-7f4d2f,georgia-pacific-a1ec53,10.0,(A8306) Sludge Blend Tank,Brewton Mill,Georgia Pacific
6,20191008-10db95,0.75,0.66630,0.751657,25,165,0,3,0,0,...,0,job-1de4c3,nekoosa-pulp-and-paper-mill-226e84,blow-tank-330023,bottom-section-tank-wall-068ed9,domtar-745a5a,18.0,Blow Tank,Nekoosa Pulp and Paper Mill,Domtar
7,20191008-e60a0f,0.75,0.39573,0.680897,82,3917,151,3909,3011,810,...,0,job-1de4c3,nekoosa-pulp-and-paper-mill-226e84,blow-tank-330023,tank-wall-6f42d7,domtar-745a5a,26.0,Blow Tank,Nekoosa Pulp and Paper Mill,Domtar


In [94]:
set(tanks_df['links.unit.data.portal.name'])

{'#1 Acid Tank',
 '#3 Black Liquor Storage Tank',
 '#4 Black Liquor Storage Tank',
 '#4 Weak Black Liquor Tank',
 '(12305) Neutralization Acid Tank',
 '(14304) 93% Sulfuric Acid Tank',
 '(2307) Blow Tank',
 '(3307) Tank',
 '(3309) #5 Filtrate Tank',
 '(3385) Tank North BMP Spill',
 '(4311) Sulfuric Acid Tank',
 '(7303) Sulfide Tank',
 '(8302) Tank',
 '(8310) Tank',
 '(A2304) Blow Tank',
 '(A2306) Blow Tank',
 '(A4303) East Acid Tank',
 '(A8305) Tank Blend',
 '(A8306) Sludge Blend Tank',
 'Accumulater Tank',
 'Accumulator Tank',
 'Black Liquor Flash Tank',
 'Blow Tank',
 'Boiler Collect Tank',
 'Caustic Soda Tank',
 'Demineralized Water Storage Tank',
 'Dump Tank',
 'Elevated Seawater Tank',
 'Evap Collect Tank',
 'Firewater Tank 54073',
 'Fuel Tank 2',
 'Green Liquor Clarifier Tank',
 'HD Tank #10',
 'Hot Water Tank 205-042',
 'Mini Soap Tank',
 'Pure Condensate Collection Tank',
 'Spill Tank',
 'Sulfuric Acid Tank M260635',
 'Sulfuric Acid Tank M370054',
 'Tank 9',
 'Water Tank',
 'We

In [93]:
tanks_df.shape

(53, 27)

In [91]:
set(tanks_df['links.component.data.portal.shell_diameter'])

{0.0,
 1.0,
 nan,
 10.0,
 12.0,
 15.0,
 16.0,
 18.0,
 18.5,
 20.0,
 25.0,
 26.0,
 28.0,
 30.0,
 36.0,
 38.0,
 40.0,
 44.0,
 45.0,
 50.0,
 55.0,
 60.0,
 67.0,
 80.0,
 100.0}

In [95]:
tanks_df.to_csv('../data/outputs/paper_mill_tanks.csv')